# Properties for ML models

This tutorial demonstrates how to use ORCA together with the ORCA Python interface (OPI) to calculate electronic properties relevant for, e.g., training machine-learned interatomic potentials (MLIPs). As an example, we will use the reference level &omega;B97M-V/def2-TZVPD and the settings used for generating the [OMol25](https://arxiv.org/abs/2505.08762) dataset. As an exemplary molecule, we use 4-methylpyrazole.

In this notebook we will:
1.  Import the required Python dependencies.
2.  Define a working directory.
3.  Set up the input structure.
4.  Define the calculation settings.
5.  Perform the ORCA calculation.
6.  Check for proper termination and convergence.
7.  Process the results.

## Step 1: Import Dependencies

We start by importing the modules needed for:
- Interfacing with ORCA input/output
- Numerical calculations and data handling
- Plotting results


In [1]:
from pathlib import Path
import shutil

# > pandas and numpy for data handling
import pandas as pd
import numpy as np

# > OPI imports for performing ORCA calculations and reading output
from opi.core import Calculator
from opi.output.core import Output
from opi.input.simple_keywords import BasisSet, AtomicCharge, Scf, \
    AuxBasisSet, Approximation, Dft, Task, OutputControl, Grid, ShellType
from opi.input.blocks import BlockScf
from opi.input.structures.structure import Structure
from opi.utils.units import AU_TO_EV

# > for visualization of molecules
import py3Dmol

## Step 2: Working directory

We define a subfolder `RUN` in which the actual ORCA calculations will take place.

In [2]:
# > Calculation is performed in `RUN`
working_dir = Path("RUN")
# > The `working_dir`is automatically (re-)created
shutil.rmtree(working_dir, ignore_errors=True)
working_dir.mkdir()

## Step 3: Setup the Input Structure

We define the structure with Cartesian coordinates in angstrom and visualize it:


In [3]:
# > Define cartesian coordinates in angstrom as Python string 
xyz_data = """\
12

N  1.3237  0.6845 -0.0002
N  1.4406 -0.6544 -0.0001
C -0.7492 -0.0043  0.0001
C  0.0327  1.1249  0.0002
C -2.2189 -0.0810 -0.0002
C  0.1711 -1.0697  0.0001
H -0.2057  2.1791  0.0003
H -2.6522  0.6920 -0.6433
H -2.5652 -1.0527 -0.3670
H -2.6091  0.0584  1.0131
H -0.0275 -2.1321  0.0003
H  2.1699  1.2401 -0.0003\n
"""
# > Visualize the input structure
view = py3Dmol.view(width=400, height=400)
view.addModel(xyz_data, 'xyz')
view.setStyle({}, {'stick': {}, 'sphere': {'scale': 0.3}})
view.zoomTo()
view.show()

# > Write the input structure to a file
with open(working_dir / "struc.xyz","w") as f:
    f.write(xyz_data)
# > Read structure into object
structure = Structure.from_xyz(working_dir / "struc.xyz")

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Step 4: Define the Calculation Settings
Next, we have to define the settings used in the ORCA calculation. This can be done by defining a calculator and adding simple keywords to it:

In [4]:
def setup_calc(basename : str, working_dir: Path, structure: Structure, ncores: int = 4) -> Calculator:
    # > Set up a Calculator object, the basename for the ORCA calculation is set to LED
    calc = Calculator(basename=basename, working_dir=working_dir)
    # > Assign structure to calculator
    calc.structure = structure

    # > Define a level of theory: wB97M-V/def2-TZVPPD
    sk_list = [
        Dft.WB97M_V, # > wB97M-V method
        BasisSet.DEF2_TZVPD, # > def2-TZVPD basis set
        AuxBasisSet.DEF2_J, # > auxiliary basis set
        Scf.TIGHTSCF, # > tight SCF
        Scf.DIIS, # > DIIS (used also by default)
        Scf.NOSOSCF, # > Disable approximated second order SCF
        ShellType.UKS, # > Always do unrestricted KS-DFT
        AtomicCharge.ALLPOP, # > Turn on all atomic population analysis
        Approximation.RIJCOSX, # > Use RIJ and COSX
        Grid.DEFGRID3, # > Large grid for exchange-correlation and COSX
        Task.ENGRAD, # > Calculate energy and gradient
        OutputControl.PRINTGAP, # > Print HOMO-LUMO gap
    ]

    # > Use simple keywords in calculator
    calc.input.add_simple_keywords(*sk_list)

    # > Add additional convergence thresholds
    calc.input.add_blocks(BlockScf(thresh=1e-12, tcut=1e-13))

    # Define number of CPUs for the calcualtion
    calc.input.ncores = ncores # > 4 CPUs for this ORCA run

    return calc

calc = setup_calc("single_point", working_dir=working_dir, structure=structure)

## Step 5: Perform the ORCA calculation

To perform the calculation, we have to write the input in ORCA format first, and then run the job by using the `run()` function of the calculator class. The output of ORCA can be written to the calculator with its `get_output()` function.

In [5]:
def run_calc(calc: Calculator) -> Output:
    # > Write the ORCA input file
    calc.write_input()
    # > Run the ORCA calculation
    print("Running ORCA calculation ...", end="")
    calc.run()
    print("   Done")

    # > Get the output object
    output = calc.get_output()
    
    return output

output = run_calc(calc)

Running ORCA calculation ...   Done


## Step 6: Check for Proper Termination and Convergence
When performing automated computations, a termination and convergence check should always be included and, when computing larger data sets, failed convergence should be handled. A simple check could look like:

In [6]:
def check_and_parse_output(output: Output):
    # > Check for proper termination of ORCA
    status = output.terminated_normally()
    if not status:
        # > ORCA did not terminate normally
        raise RuntimeError("ORCA did not terminate normally. Please check RUN/single_point.out")
    else:
        # > ORCA did terminate normally so we can parse the output
        output.parse()

    # > Now check for convergence of the SCF
    if output.results_properties.geometries[0].single_point_data.converged:
        print("SCF CONVERGED")
    else:
        raise RuntimeError("SCF DID NOT CONVERGE")
    
check_and_parse_output(output)

SCF CONVERGED


## Step 7: Process the Results
After a successful calculation, we can access the results and store them. For example:

In [ ]:
def process_output(output: Output) -> dict:
    # > Conversion factor for Hartree -> eV
    h_to_eV = 27.2114
    # > Conversion factor for Bohr -> Angstrom
    bohr_to_A = 0.529177

    # > Energy and gradient information
    # > Total energy in eV
    Etot = output.results_properties.geometries[0].single_point_data.finalenergy*h_to_eV
    # > VV10 energy part in eV
    # > Note that VV10 is self-consistent in gradient computations and thus ecnl is used
    E_VV10 = output.results_properties.geometries[0].dft_energy.ecnl*h_to_eV
    # > Gradient in eV/Bohr (list of size n_atoms containing one list per atom containing three entries)
    Grad = output.results_properties.geometries[0].nuclear_gradient[0].grad
    Grad = [[x * (h_to_eV/bohr_to_A) for x in nuc_grad] for nuc_grad in Grad]

    # > General information
    # > Charge of molecule (float value)
    tot_chrg = output.results_properties.calculation_info.charge
    # > Spin of molecule (float value)
    tot_spin = output.results_properties.calculation_info.mult
    # > Number of atoms (float value)
    n_at = output.results_properties.calculation_info.numofatoms
    # > Number of electrons (float value)
    n_el = output.results_properties.calculation_info.numofelectrons
    # > Number of basis functions (float value)
    n_bf = output.results_properties.calculation_info.numofbasisfuncts
    # > Number of alpha electrons (float value)
    n_el_alp = output.results_properties.geometries[0].dft_energy.nalphael
    # > Number of beta electrons (float value)
    n_el_beta = output.results_properties.geometries[0].dft_energy.nbetael

    # > Population analysis
    # > Loewdin charges (list of size n_atoms containing lists with 3 entries)
    chrgs_lowedin = output.results_properties.geometries[0].loewdin_population_analysis[0].atomiccharges
    # > Mulliken charges (list of size n_atoms containing lists with 3 entries)
    chrgs_mulliken = output.results_properties.geometries[0].mulliken_population_analysis[0].atomiccharges
    # > Mayer bond order (list of size of the number of bonds)
    bond_order = output.results_properties.geometries[0].mayer_population_analysis[0].bondorders
    # > Mayer's total valence (list of size n_atoms)
    va_mayer = output.results_properties.geometries[0].mayer_population_analysis[0].va

    # > Dipole moments
    # > Total dipole moments (list with 3 entries: x, y, z)
    dip_tot = output.results_properties.geometries[0].dipole_moment[0].dipoletotal
    # > Magnitude of dipole (float value)
    dip_mag = output.results_properties.geometries[0].dipole_moment[0].dipolemagnitude
    # > Electronic dipole contributions (list with 3 entries: x, y, z)
    dip_el = output.results_properties.geometries[0].dipole_moment[0].dipoleeleccontrib
    # > Nuclear dipole contributions (list with 3 entries: x, y, z)
    dip_nu = output.results_properties.geometries[0].dipole_moment[0].dipolenuccontrib

    # > MO handling
    mos = output.get_mos()  # saves a list of MOs with size n_atoms
    mo_energies = []
    mo_occupancy = []
    # > Get the alpha-spin MOs. Similarly, the beta-spin MOs are returned with mos["beta"].
    # > For closed shell calculations, use mos["mos"] 
    for mo in mos["alpha"]:
        mo_energies.append(mo.orbitalenergy)  # Extract the MO energies
        mo_occupancy.append(mo.occupancy)  # Extract the occupation numbers
    mo_energies = np.array(mo_energies)  # Make float list
    mo_occupancy = np.array(mo_occupancy)  # Make float list

    HOMO = output.get_homo()
    HOMO_energy = HOMO.orbitalenergy * AU_TO_EV
    LUMO = output.get_lumo()
    LUMO_energy = LUMO.orbitalenergy * AU_TO_EV

    summary_data = {
    "Total Energy [eV]": [Etot],
    "Gradient [ev/A]": [str(Grad[0][0])[:10]+', '+str(Grad[1][0])[:10]+', '+str(Grad[2][0])[:10]],
    "VV10 Energy [eV]": [E_VV10],
    "Dipole Magnitude": [dip_mag],
    "Total Charge": [tot_chrg],
    "Spin Multiplicity": [tot_spin],
    "Number of Atoms": [n_at],
    "Number of Electrons": [n_el],
    "Alpha Electrons": [n_el_alp],
    "Beta Electrons": [n_el_beta],
    "Basis Functions": [n_bf],
    "HOMO Energy [eV]": [HOMO_energy],
    "LUMO Energy [eV]": [LUMO_energy],
    }
    
    return summary_data

summary_data = process_output(output)

{'alpha': [MO(mocoefficients=[0.4499650820427518, 0.6267665116425329, 0.032420569327877055, -0.030865482112867184, 0.07439870397468194, 8.41915129246138e-06, -0.00023591067117614074, -0.0003860208679022895, -3.263823075287741e-05, 0.0019077250145464633, 0.0010952756473646618, 0.0001561400015239913, -0.002748416030979819, -0.014761935293227373, -0.0002641206532421062, -1.2091208468638434e-06, -1.3790405124567266e-06, 4.3393700723051655e-05, 1.6964029806726523e-05, 0.0003374382982692174, 5.9872643819581905e-06, 5.536529890484303e-06, -7.361827443586692e-05, 0.0003050706513435446, -1.0702220475189017e-06, -6.105155976060096e-05, -4.532240959700846e-05, -3.5873765346436046e-07, -1.0739374196678902e-06, 1.875540088480881e-05, -0.00014758646406459072, 0.01973808979326845, -0.0039196117376644945, -8.409808271515265e-05, -4.691429843286324e-05, -0.002574019341596773, -0.00028737330356450784, 0.0007838291043336967, 0.001121283561539007, -0.0002985766995541424, 0.002968105573048185, -0.025825071

In the following, the resulting values are shown with pandas:

In [14]:
df_summary = pd.DataFrame(summary_data, index=[""])
df_summary.T

,
Total Energy [eV],-7224.999588
Gradient [ev/A],"-0.2614030, -0.2177816, -0.0299155"
VV10 Energy [eV],4.606444
Dipole Magnitude,0.942633
Total Charge,0
Spin Multiplicity,1
Number of Atoms,12
Number of Electrons,44
Alpha Electrons,22
Beta Electrons,22
